In [1]:
import numpy as np
import pandas as pd

# Loading Data

Let's start by loading a large geographic dataset. In this notebook we will be using the Dublin Bus dataset as loaded and prepared on another [repository](https://github.com/joaofig/dublin-bus "Dublin Buses"). You just need the very first notebook to download and prepare the data, so we can use it here. Please be patient as it may take some time. Once you have the data file, please copy it to the data folder.

Note: Make sure you have the pyarrow package installed for the code below to work.

In [2]:
columns_to_read = ['Lon', 'Lat']
df = pd.read_parquet("data/sir010113-310113.parquet", columns=columns_to_read)

Note that, for convenience, the data frame is read in with a reset index, which may be quite useful in the forthcoming computations.

# Brute-Force
We start by using a brute-force approach to finding all the points whithin a 100 meter radius from arbitrarily selected locations. Two such locations were selected from the Dublin map, curtesy of Google maps: the University College and the Guiness Storehouse.

The brute-force approach implies calculating the distance between these two points and all the other 44 million points from the Dublin Bus dataset. Once the distances are calculated, we can simply select the ones that are whithin the 100 meter radius.

In [3]:
from geo.geomath import vec_haversine, num_haversine

Define the two locations for which we want to query all the sampled points within a 100 meter radius.

In [4]:
uni_col_lat = 53.3277162
uni_col_lon = -6.2672435
uni_col = np.array([[uni_col_lat, uni_col_lon]])

In [5]:
guiness_lat = 53.3428673
guiness_lon = -6.2717738
guiness = np.array([[guiness_lat, guiness_lon]])

Now we extract all the latitudes and longitudes to specific NumPy arrays for easier reuse.

In [6]:
lats = df['Lat'].to_numpy()
lons = df['Lon'].to_numpy()

In [7]:
uni_col_dx = vec_haversine(lats, lons, uni_col_lat, uni_col_lon)

In [8]:
guiness_dx = vec_haversine(lats, lons, guiness_lat, guiness_lon)

In [9]:
uni_col_dx[uni_col_dx <= 100.0]

array([], dtype=float64)

In [10]:
guiness_dx[guiness_dx <= 100.0]

array([24.87038276, 24.87038276, 65.76017745, ..., 47.52239545,
       37.78872007, 47.52239545])

In [11]:
guiness_dx[guiness_dx <= 100.0].shape

(97867,)

# Triangle Inequality
The triangle inequality query is implemented by the GeoQuery clas. It uses an interface that is quite similar to the BallTree class (see below).

In [29]:
from geo.geoquery import GeoSpoke

In [30]:
positions = df[['Lat', 'Lon']].to_numpy()

In [31]:
%%timeit -r1 -n1
geo_query = GeoSpoke(positions)

29.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [32]:
ind = geo_query.query_radius(guiness, r=100.0)

In [33]:
ind.shape

(0,)

# Building a BallTree

In this section, we will create a BallTree to perform fast searches on our Dublin geographic data. The queries we will look into are *k-nearest neighbors* and *neighbors within a given radius*. Both the tree object and the distance metric object live in the scikit-learn *neighbors* namespace. Let's import those first.

In [18]:
from sklearn.neighbors import BallTree

Before we build the tree, we must select the latitude and longitude columns of the data frame. The distance measure for geographic coordinates is the *haversine distance* and the DistanceMetric class requires that we feed the locations as an array of latitude and longitude in radians.

In [19]:
positions = np.radians(df[['Lat', 'Lon']].to_numpy())

Now we can create the BallTree using the *positions* array. Please be patient as the next line may take some time to run.

In [20]:
%%timeit -r1 -n1
tree = BallTree(positions, metric="haversine")

In [21]:
import math

In [22]:
earth_radius = 6371000.0

In [23]:
guiness_lat_r = math.radians(guiness_lat) 
guiness_lon_r = math.radians(guiness_lon)

In [24]:
guiness = np.array([guiness_lat_r, guiness_lon_r]).reshape(1, -1)

In [ ]:
# dist, ind = tree.query(guiness, k=100) 

In [25]:
ind = tree.query_radius(guiness, r=100.0 / earth_radius) 

In [28]:
ind[0].shape

(97867,)

In [ ]:
dist0 = vec_haversine(lats, lons, 0.0, 0.0)

In [ ]:
dist1 = vec_haversine(lats, lons, 90.0, 0.0)

In [ ]:
idx0 = np.argsort(dist0)

In [ ]:
idx1 = np.argsort(dist1)

In [ ]:
sorted0 = dist0[idx0]

In [ ]:
sorted1 = dist1[idx1]

In [ ]:
gui0 = num_haversine(guiness_lat, guiness_lon, 0.0, 0.0)
gui1 = num_haversine(guiness_lat, guiness_lon, 90.0, 0.0)

In [ ]:
gui0, gui1

In [ ]:
np.searchsorted(sorted0, gui0)

In [ ]:
np.searchsorted(sorted1, gui1)

In [ ]:
sorted0[18151963], dist0[idx0[18151963]]

In [ ]:
sorted1[25614622], dist1[idx1[25614622]]

# Range Search

Perform a range search using the spoke method

In [ ]:
i0 = np.searchsorted(sorted0, gui0 - 100.0)
i1 = np.searchsorted(sorted0, gui0 + 100.0)
match0 = idx0[i0:i1+1]

In [ ]:
i0 = np.searchsorted(sorted1, gui1 - 100.0)
i1 = np.searchsorted(sorted1, gui1 + 100.0)
match1 = idx1[i0:i1+1]

In [ ]:
match0.shape

In [ ]:
match1.shape

In [ ]:
intersect = np.intersect1d(match0, match1)

In [ ]:
intersect.shape

In [ ]:
radii = vec_haversine(lats[intersect], lons[intersect], guiness_lat, guiness_lon)

In [ ]:
valid_r = radii <= 100.0

In [ ]:
valid_r.sum()

In [ ]:
intersect[valid_r]